<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#이것들은-향후-예보-데이터가-아닌-이미-발생한-당일날의-기상-데이터들임!" data-toc-modified-id="이것들은-향후-예보-데이터가-아닌-이미-발생한-당일날의-기상-데이터들임!-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>이것들은 향후 예보 데이터가 아닌 이미 발생한 당일날의 기상 데이터들임!</a></span><ul class="toc-item"><li><span><a href="#2020-12-31일까지의-데이터만-수집하는-코드" data-toc-modified-id="2020-12-31일까지의-데이터만-수집하는-코드-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>2020-12-31일까지의 데이터만 수집하는 코드</a></span></li><li><span><a href="#실시간-데이터-수집하기-용-코드" data-toc-modified-id="실시간-데이터-수집하기-용-코드-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>실시간 데이터 수집하기 용 코드</a></span></li></ul></li></ul></div>

# 이것들은 향후 예보 데이터가 아닌 이미 발생한 당일날의 기상 데이터들임!

In [1]:
import requests
from bs4 import BeautifulSoup

- 필요한 파라미터들
    - stnNm : 지점명
    - tm : 시간
    - avgTa : 평균 기온
    - minTa : 최저 기온
    - maxTa : 최고 기온
    - hr1MaxRn : 1시간 최다 강수량
    - sumRn : 일강수량
    - maxWs : 최대 풍속
    - avgWs : 평균 풍속
    - minRhm : 최소 상대습도
    - avgRhm : 평균 상대습도
    - hr1MaxIcsr : 1시간 최다 일사량
    - sumGsr : 합계 일사

In [2]:
import pandas as pd
import numpy as np
import datetime
import math

## 2020-12-31일까지의 데이터만 수집하는 코드

In [5]:
time_delta = pd.Timestamp('2020-12-31') - pd.Timestamp('2013-01-07')
time_delta = time_delta.days + 1
max_pages = math.ceil((time_delta / 999) + 1)

# 전체 데이터셋 담을 데이터프레임
all_datasets = pd.DataFrame()


# 5페이지 까지 Loop
pages = [n for n in range(1, max_pages)]

for page in pages:
    res = requests.get(f'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=j%2FJXmL%2BFxwnNYqN%2FyoLJSfJx3ioQV1HnmM9E7b%2FaGLjLv51g0vZSGQjk0UVyJmGZckzK7Cm8Jds6G42cqdkX0w%3D%3D&numOfRows=999&pageNo={page}&dataCd=ASOS&dateCd=DAY&startDt=20130107&endDt=20201231&stnIds=133')
    soup = BeautifulSoup(res.content, 'html.parser')

    # 서브 데이터프레임으로 만들 dict 정의
    data_dict = {}

    # 각 변수 값들 담을 list
    region_lst, datetime_lst = [], []
    avg_temp_lst, min_temp_lst = [], []
    min_temp_lst, max_temp_lst = [], []
    max_1h_rain_lst, sum_rain_lst = [], []
    avg_wind_lst, max_wind_lst = [], []
    avg_humid_lst, min_humid_lst = [], []
    max_1h_sun_lst, sum_sun_lst = [], []

    contents = soup.select('item')
    # 각 페이지에서 변수 별 데이터 수집
    for content in contents:
        region = content.select_one('stnnm').get_text()
        datetime = content.select_one('tm').get_text()
        avg_temp = content.select_one('avgta').get_text()
        min_temp = content.select_one('minta').get_text()
        max_temp = content.select_one('maxta').get_text()
        max_1h_rain = content.select_one('hr1maxrn').get_text()
        sum_rain = content.select_one('sumrn').get_text()
        avg_wind = content.select_one('avgws').get_text()
        max_wind = content.select_one('maxws').get_text()
        avg_humid = content.select_one('avgrhm')
        min_humid = content.select_one('minrhm').get_text()
        max_1h_sun = content.select_one('hr1maxicsr').get_text()
        sum_sun = content.select_one('sumgsr').get_text()

        region_lst.append(region)
        datetime_lst.append(datetime)
        avg_temp_lst.append(avg_temp)
        min_temp_lst.append(min_temp)
        max_temp_lst.append(max_temp)
        max_1h_rain_lst.append(max_1h_rain)
        sum_rain_lst.append(sum_rain)
        avg_wind_lst.append(avg_wind)
        max_wind_lst.append(max_wind)
        avg_humid_lst.append(avg_humid)
        min_humid_lst.append(min_humid)
        max_1h_sun_lst.append(max_1h_sun)
        sum_sun_lst.append(sum_sun)

    data_dict['지역'] =  region_lst
    data_dict['시간'] = datetime_lst
    data_dict['평균기온'] = avg_temp_lst
    data_dict['최저기온'] = min_temp_lst
    data_dict['최고기온'] = max_temp_lst
    data_dict['1시간최다강수량'] = max_1h_rain_lst
    data_dict['일강수량'] = sum_rain_lst
    data_dict['평균풍속'] = avg_wind_lst
    data_dict['최대풍속'] = max_wind_lst
    data_dict['평균상대습도'] = avg_humid_lst
    data_dict['최소상대습도'] = min_humid_lst
    data_dict['1시간최다일사량'] = max_1h_sun_lst
    data_dict['일사량'] = sum_sun_lst
    
    # 1페이지 당 서브 데이터프레임 만들기
    sub_dataframe = pd.DataFrame(data_dict)
    print(f'## {page}페이지 데이터 크롤링 완료!')
    
    # 전체 데이터프레임에 concat 하기
    all_datasets = pd.concat([all_datasets, sub_dataframe], axis=0)
    print(f'## {page}페이지 전체 데이터셋에 concat 완료!')
    print()

## 1페이지 데이터 크롤링 완료!
## 1페이지 전체 데이터셋에 concat 완료!

## 2페이지 데이터 크롤링 완료!
## 2페이지 전체 데이터셋에 concat 완료!

## 3페이지 데이터 크롤링 완료!
## 3페이지 전체 데이터셋에 concat 완료!



In [6]:
all_datasets

,지역,시간,평균기온,최저기온,최고기온,1시간최다강수량,일강수량,평균풍속,최대풍속,평균상대습도,최소상대습도,1시간최다일사량,일사량
0,대전,2013-01-07,-5.0,-10.7,1.0,,,1.0,2.7,[66.4],32,2.06,10.99
1,대전,2013-01-08,-4.2,-9.2,1.9,,,1.1,3.0,[68.0],43,1.87,11.16
2,대전,2013-01-09,-6.3,-9.7,-1.9,,,1.9,5.2,[59.5],39,2.13,12.34
3,대전,2013-01-10,-7.9,-13.8,-1.5,,,1.1,3.1,[54.9],29,2.15,12.69
4,대전,2013-01-11,-5.3,-12.8,1.9,,,0.9,1.9,[70.0],50,1.99,11.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,대전,2020-12-27,3.0,-2.0,8.8,,,0.6,1.6,[78.4],56,1.24,6.02
914,대전,2020-12-28,5.4,-0.2,13.4,,,0.8,1.7,[73.4],36,1.77,9.55
915,대전,2020-12-29,1.3,-2.2,6.3,,2.5,1.5,4.2,[84.4],64,,
916,대전,2020-12-30,-7.5,-10.5,-2.2,,0.0,3.1,5.7,[63.5],49,1.34,7.59


---

## 실시간 데이터 수집하기 용 코드

- 기상 데이터는 오늘 날짜 기준으로 어제까지의 기상 데이터가 가장 최근의 데이터수집된 상태임!

In [120]:
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)
time_delta = pd.Timestamp(yesterday) - pd.Timestamp('2013-01-07')
time_delta = time_delta.days
max_pages = math.ceil((time_delta / 999) + 1)
# URL 요청시 파라미터에 넣어줄 end time
yesterday_str = yesterday.strftime('%Y%m%d')

# 전체 데이터셋 담을 데이터프레임
all_datasets = pd.DataFrame()


# 5페이지 까지 Loop
pages = [n for n in range(1, max_pages)]

for page in pages:
    res = requests.get(f'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=j%2FJXmL%2BFxwnNYqN%2FyoLJSfJx3ioQV1HnmM9E7b%2FaGLjLv51g0vZSGQjk0UVyJmGZckzK7Cm8Jds6G42cqdkX0w%3D%3D&numOfRows=999&pageNo={page}&dataCd=ASOS&dateCd=DAY&startDt=20130107&endDt={yesterday_str}&stnIds=133')
    soup = BeautifulSoup(res.content, 'html.parser')

    # 서브 데이터프레임으로 만들 dict 정의
    data_dict = {}

    # 각 변수 값들 담을 list
    region_lst, datetime_lst = [], []
    avg_temp_lst, min_temp_lst = [], []
    min_temp_lst, max_temp_lst = [], []
    max_1h_rain_lst, sum_rain_lst = [], []
    avg_wind_lst, max_wind_lst = [], []
    avg_humid_lst, min_humid_lst = [], []
    max_1h_sun_lst, sum_sun_lst = [], []

    contents = soup.select('item')
    # 각 페이지에서 변수 별 데이터 수집
    for content in contents:
        region = content.select_one('stnnm').get_text()
        datetime = content.select_one('tm').get_text()
        avg_temp = content.select_one('avgta').get_text()
        min_temp = content.select_one('minta').get_text()
        max_temp = content.select_one('maxta').get_text()
        max_1h_rain = content.select_one('hr1maxrn').get_text()
        sum_rain = content.select_one('sumrn').get_text()
        avg_wind = content.select_one('avgws').get_text()
        max_wind = content.select_one('maxws').get_text()
        avg_humid = content.select_one('avgrhm')
        min_humid = content.select_one('minrhm').get_text()
        max_1h_sun = content.select_one('hr1maxicsr').get_text()
        sum_sun = content.select_one('sumgsr').get_text()

        region_lst.append(region)
        datetime_lst.append(datetime)
        avg_temp_lst.append(avg_temp)
        min_temp_lst.append(min_temp)
        max_temp_lst.append(max_temp)
        max_1h_rain_lst.append(max_1h_rain)
        sum_rain_lst.append(sum_rain)
        avg_wind_lst.append(avg_wind)
        max_wind_lst.append(max_wind)
        avg_humid_lst.append(avg_humid)
        min_humid_lst.append(min_humid)
        max_1h_sun_lst.append(max_1h_sun)
        sum_sun_lst.append(sum_sun)

    data_dict['지역'] =  region_lst
    data_dict['시간'] = datetime_lst
    data_dict['평균기온'] = avg_temp_lst
    data_dict['최저기온'] = min_temp_lst
    data_dict['최고기온'] = max_temp_lst
    data_dict['1시간최다강수량'] = max_1h_rain_lst
    data_dict['일강수량'] = sum_rain_lst
    data_dict['평균풍속'] = avg_wind_lst
    data_dict['최대풍속'] = max_wind_lst
    data_dict['평균상대습도'] = avg_humid_lst
    data_dict['최소상대습도'] = min_humid_lst
    data_dict['1시간최다일사량'] = max_1h_sun_lst
    data_dict['일사량'] = sum_sun_lst
    
    # 1페이지 당 서브 데이터프레임 만들기
    sub_dataframe = pd.DataFrame(data_dict)
    print(f'## {page}페이지 데이터 크롤링 완료!')
    
    # 전체 데이터프레임에 concat 하기
    all_datasets = pd.concat([all_datasets, sub_dataframe], axis=0)
    print(f'## {page}페이지 전체 데이터셋에 concat 완료!')
    print()
    

## 1페이지 데이터 크롤링 완료!
## 1페이지 전체 데이터셋에 concat 완료!
## 2페이지 데이터 크롤링 완료!
## 2페이지 전체 데이터셋에 concat 완료!
## 3페이지 데이터 크롤링 완료!
## 3페이지 전체 데이터셋에 concat 완료!
